# Import dependencies

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils

# Load the MNIST Dataset

In [ ]:
# Parameters for MNIST dataset
img_rows, img_cols = 28, 28
number_of_data = 1000

# Load MNIST Dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), img_rows * img_cols))
x_test = x_test.reshape((len(x_test), img_rows * img_cols))
x_train = x_train[:number_of_data]
x_test = x_test[:number_of_data]

print(x_train.shape)
print(x_test.shape)

# Visualize MNIST dataset
import matplotlib.pyplot as plt
%matplotlib inline

n = 10  # how many digits we will display
plt.figure(figsize=(10, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Autoencoder
<img align="left" width="40%" src="images/autoencoder.png"/>

## Build

In [ ]:
# Parameters for autoencoder
nb_visible = img_rows * img_cols
nb_hidden = 100

autoencoder = Sequential()

autoencoder.add(Dense(nb_hidden, input_shape=(nb_visible,), activation='relu'))
autoencoder.add(Dense(nb_visible, activation='sigmoid'))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

## Train

In [ ]:
# Settings
batch_size = 128
nb_epoch = 50

# Train
autoencoder.fit(x_train, x_train,
                epochs=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1,
                validation_data=(x_test, x_test))

## Evaluate

In [ ]:
evaluation = autoencoder.evaluate(x_test, x_test, batch_size=batch_size, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f' % (evaluation))

## Visualize Reconstruction

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Decode test images 
decoded_imgs = autoencoder.predict(x_test)

n = 10  # how many digits we will display
plt.figure(figsize=(10, 2))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Visualize Denoising

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Decode test images 
x_test_corrupted = x_test + 0.4 * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)

    
x_test_corrupted = np.array(x_test_corrupted)
decoded_imgs = autoencoder.predict(x_test_corrupted)

n = 10  # how many digits we will display
plt.figure(figsize=(10, 2))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_corrupted[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Convolutional Autoencoder
<img align="left" width="90%" src="images/cnn_autoencoder.png"/>

## Load MNIST data in 2D

In [ ]:
# Parameters for MNIST dataset
img_rows, img_cols, img_chns = 28, 28, 1
number_of_data = 1000
nb_classes = 10

# Load MNIST Dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), img_rows, img_cols, img_chns))
x_test = x_test.reshape((len(x_test), img_rows, img_cols, img_chns))
x_train = x_train[:number_of_data]
x_test = x_test[:number_of_data]
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
y_train = y_train[:number_of_data]
y_test = y_test[:number_of_data]

print(x_train.shape)
print(x_test.shape)

## Build

In [ ]:
# Settings
pool_size = (2, 2)                  # size of pooling area for max pooling
input_shape = (img_rows, img_cols, img_chns)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))

# at this point the representation is (7, 7, 32)

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(UpSampling2D(pool_size))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(UpSampling2D(pool_size))
model.add(Conv2D(1, (3, 3), padding='same', activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

## Training

In [ ]:
# Settings
batch_size = 128
nb_epoch = 15

# Train
model.fit(x_train, x_train,
                epochs=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1,
                validation_data=(x_test, x_test))

## Evaluate

In [ ]:
evaluation = model.evaluate(x_test, x_test, batch_size=batch_size, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f' % (evaluation))

## Visualize Reconstruction

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Decode test images 
decoded_imgs = model.predict(x_test)

n = 10  # how many digits we will display
plt.figure(figsize=(10, 2))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Visualize Denoising

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Decode test images 
#x_test_corrupted = x_test[10:]
x_test_corrupted = x_test + 0.4 * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)

    
x_test_corrupted = np.array(x_test_corrupted)
decoded_imgs = model.predict(x_test_corrupted)

n = 10  # how many digits we will display
plt.figure(figsize=(10, 2))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_corrupted[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Supplementary - Use Feature from Autoencoder


## Create model with functional Keras

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))
x = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()
autoencoder.fit(x_train, x_train,
                epochs=15,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

## Get encoded data

In [ ]:
encoder = Model(input_img, encoded)

encoded_imgs = encoder.predict(x_train)
print 'Encode image shape:', encoded_imgs.shape
encoded_imgs = encoded_imgs.reshape(encoded_imgs.shape[0],-1) # flatten 
print 'Encode image shape after reshape:', encoded_imgs.shape

encoded_test = encoder.predict(x_test)
encoded_test = encoded_test.reshape(encoded_test.shape[0],-1) # flatten 
print 'Encode image shape:', encoded_test.shape
print 'Encode image shape after reshape:', encoded_test.shape



## Train a Classifier

In [ ]:
model = Sequential()
model.add(Dense(units=200, input_dim=encoded_imgs.shape[1], kernel_initializer='normal', activation='sigmoid'))
model.add(Dense(units=10, input_dim=200, kernel_initializer='normal', activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

batch_size = 128
epochs = 10

history = model.fit(encoded_imgs, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

## Evaluate

In [ ]:
evaluation = model.evaluate(encoded_test, y_test, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))